#### Names of people in the group

Please write the names of the people in your group in the next cell.

Nick Askari

Simen Peder Stang

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ff55483d-e868-4a95-b63b-773300c5d893/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [ ]:
 %fs ls /user/hive/warehouse

path name size modificationTime dbfs:/user/hive/warehouse/badges/ badges/ 0 0 dbfs:/user/hive/warehouse/comments/ comments/ 0 0 dbfs:/user/hive/warehouse/posts/ posts/ 0 0 dbfs:/user/hive/warehouse/users/ users/ 0 0

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [ ]:
from pyspark.sql import SparkSession
from typing import Any

# Initialize Spark session (assuming it's not already initialized)
spark = SparkSession.builder.appName("Spark SQL Helper Functions").getOrCreate()

def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    ## To-do!
    df.createOrReplaceTempView("df")
    
    # Execute the query
    result_df = spark.sql(query)
    
    # Return the content of the first column of the first row
    return result_df.collect()[0][0]

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    ## To-do!
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    
    # Execute the query
    result_df = spark.sql(query)
    
    # Return the content of the first column of the first row
    return result_df.collect()[0][0]

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [ ]:
q1 = '''SELECT * 
        FROM df 
        ORDER BY CreationDate DESC 
        LIMIT 1
'''

## To-do!
q2 = "SELECT COUNT(Id) AS NumUsers FROM df"

## To-do!
q3 = '''SELECT OwnerUserId, COUNT(*) AS NumAnswers 
        FROM df
        WHERE PostTypeId=2 
        GROUP BY OwnerUserId 
        ORDER BY NumAnswers DESC 
        LIMIT 1
'''

## To-do!
q4 = "SELECT COUNT(*) AS NumQuestions FROM df WHERE PostTypeId = 1"

## To-do!
q5 = '''SELECT DisplayName, COUNT(*) AS TotalOccurrences
        FROM df1
        JOIN df2 ON df1.Id=df2.UserId
        GROUP BY DisplayName
        ORDER BY TotalOccurrences DESC
        LIMIT 1
'''

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 7.333s

OK
Out[80]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

Your answers...

*1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'?*

Tge differences between these three lies mostly in their level of abstraction and type safety. An RDD in spark is immutable distributed collection of objects. Moreover each RDD is split into different partitions. This way they can be computed with different nodes of the cluster. Hence these objects, can be computed in parallell. Also like mentioned in the previous task there spark incorporates lineage with these RDD's, making it easier to recompute lost data for instance. The latter is a key distinguisher. In an RDD you can have structured and unstructured dataformats. RDDs are the lowest-level abstraction provided by Spark, offering full control over data and execution, but without the optimizations that come with DataFrames and Datasets. 

A dataframe on the other hand is a distributed collection of data, with columns and rows. Frameworks like pandas is relevant here. You can typically perform some operations on a dataframe, like selection and aggregation. In other words it is very high level, and does not go to spesifics as to controlling the computation in great detail like RDD. Also dataframes in spark provide query optimization through an optimization engine called Catalyst. With dataframes structured and semi-structured dataformats are allowed. 

Datasets is a type-safe version of a DataFrame. One usually uses datasets when data requires a structure. If one desires to do SQL-queries it is also recomended to use datasets or dataframes. It combines the benefits of RDD's functional programming style with the optimization and convenience of DataFrames. 

*2. When do you suggest using RDDs instead of using DataFrames?*

If you do not require your data to be in a schema format, then RDD is the way to go. Often this is the case when you want fine grained control (low-level transformations and actions) and division of the data and over the computation involved. Also, it is suitable when operations are not easily expressed in a higher level abstraction like a DataFrame. Moreover, RDDs are better for unstructured data, and applications with complex processing on data that does not fit into tabular form, such as a stream of text. 

*3. What is the main benefit of using DataSets instead of DataFrames?*

The main benefit of using DataSets is type safety at compile time. In other words this enhances the development process by allowing you to catch a wide range of errors early in development before the code is even run (when it is compiled, not in runtime).